Entradas: Month, GHI, Wind Speed, Zenith, Azimuth, Hour Sine, Hour Cosine 

Target: Angulo

Dados de 22/07 até 18/11 

In [11]:
import pandas as pd
import pvlib
import numpy as np 
import matplotlib.pyplot as plt

In [12]:
df = pd.read_csv('data.csv') 


# otimo e theta calculados com pvlib
angulos = pd.read_csv('otimo.csv')
thetas = pd.read_csv('backtracking.csv')

# zenite e azimute do get solar position
solpos = pd.read_csv('solarposition.csv')

# Filtrar o df e pegar dados de 04/07 até 18/11 (5760 dados)
df_mes = df[(df['datetime'] >= '2019-07-22 00:00:00') & (df['datetime'] <= '2019-11-18 23:30:00')].copy()
display(df_mes)
angulos_mes = angulos[(angulos['datetime'] >= '2019-07-22 00:00:00') & (angulos['datetime'] <= '2019-11-18 23:30:00')].copy()
display(angulos_mes)
df_mes['Angulo'] = angulos_mes['angulo_otimo']
thetas_mes = thetas[(thetas['datetime_local'] >= '2019-07-22 00:00:00-03:00') & (thetas['datetime_local'] <= '2019-11-18 23:30:00-03:00')].copy()
display(thetas_mes)
solpos_mes = solpos[(solpos['datetime_local'] >= '2019-07-22 00:00:00-03:00') & (solpos['datetime_local'] <= '2019-11-18 23:30:00-03:00')].copy()
display(solpos_mes)

,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,Temperature,DNI,GHI,Solar Zenith Angle,Wind Speed,Surface Albedo,Cloud Type,Fill Flag,datetime,datetime_utc,datetime_local
9696,9696,2019,7,22,0,0,0,14.6,0,0,175.60,2.0,0.15,7,0,2019-07-22 00:00:00,2019-07-22 00:00:00+00:00,2019-07-22 00:00:00-03:00
9697,9697,2019,7,22,0,30,0,14.5,0,0,174.91,1.8,0.15,7,0,2019-07-22 00:30:00,2019-07-22 00:30:00+00:00,2019-07-22 00:30:00-03:00
9698,9698,2019,7,22,1,0,0,14.3,0,0,168.63,1.7,0.15,7,0,2019-07-22 01:00:00,2019-07-22 01:00:00+00:00,2019-07-22 01:00:00-03:00
9699,9699,2019,7,22,1,30,0,14.3,0,0,161.86,1.8,0.15,7,0,2019-07-22 01:30:00,2019-07-22 01:30:00+00:00,2019-07-22 01:30:00-03:00
9700,9700,2019,7,22,2,0,0,14.2,0,0,155.01,1.8,0.15,7,0,2019-07-22 02:00:00,2019-07-22 02:00:00+00:00,2019-07-22 02:00:00-03:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15451,15451,2019,11,18,21,30,0,17.3,0,0,124.92,3.9,0.13,3,8,2019-11-18 21:30:00,2019-11-18 21:30:00+00:00,2019-11-18 21:30:00-03:00
15452,15452,2019,11,18,22,0,0,17.2,0,0,129.18,3.9,0.13,6,8,2019-11-18 22:00:00,2019-11-18 22:00:00+00:00,2019-11-18 22:00:00-03:00
15453,15453,2019,11,18,22,30,0,17.2,0,0,132.69,3.9,0.13,6,8,2019-11-18 22:30:00,2019-11-18 22:30:00+00:00,2019-11-18 22:30:00-03:00
15454,15454,2019,11,18,23,0,0,17.1,0,0,135.28,3.8,0.13,6,8,2019-11-18 23:00:00,2019-11-18 23:00:00+00:00,2019-11-18 23:00:00-03:00


,Unnamed: 0,datetime,angulo_otimo,irradiancia
9696,9696,2019-07-22 00:00:00,15,0.0
9697,9697,2019-07-22 00:30:00,15,0.0
9698,9698,2019-07-22 01:00:00,15,0.0
9699,9699,2019-07-22 01:30:00,15,0.0
9700,9700,2019-07-22 02:00:00,15,0.0
...,...,...,...,...
15451,15451,2019-11-18 21:30:00,60,0.0
15452,15452,2019-11-18 22:00:00,15,0.0
15453,15453,2019-11-18 22:30:00,15,0.0
15454,15454,2019-11-18 23:00:00,15,0.0


,datetime_local,tracker_theta
9696,2019-07-22 00:00:00-03:00,0.0
9697,2019-07-22 00:30:00-03:00,0.0
9698,2019-07-22 01:00:00-03:00,0.0
9699,2019-07-22 01:30:00-03:00,0.0
9700,2019-07-22 02:00:00-03:00,0.0
...,...,...
15451,2019-11-18 21:30:00-03:00,0.0
15452,2019-11-18 22:00:00-03:00,0.0
15453,2019-11-18 22:30:00-03:00,0.0
15454,2019-11-18 23:00:00-03:00,0.0


,datetime_local,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
9696,2019-07-22 00:00:00-03:00,175.603772,175.603772,-85.603772,-85.603772,223.818631,-6.443682
9697,2019-07-22 00:30:00-03:00,174.914320,174.914320,-84.914320,-84.914320,128.305934,-6.444546
9698,2019-07-22 01:00:00-03:00,168.638525,168.638525,-78.638525,-78.638525,104.161072,-6.445406
9699,2019-07-22 01:30:00-03:00,161.868768,161.868768,-71.868768,-71.868768,96.488681,-6.446262
9700,2019-07-22 02:00:00-03:00,155.012959,155.012959,-65.012959,-65.012959,92.182515,-6.447115
...,...,...,...,...,...,...,...
15451,2019-11-18 21:30:00-03:00,124.927836,124.927836,-34.927836,-34.927836,221.848110,14.818052
15452,2019-11-18 22:00:00-03:00,129.190369,129.190369,-39.190369,-39.190369,214.765982,14.813566
15453,2019-11-18 22:30:00-03:00,132.701519,132.701519,-42.701519,-42.701519,206.627335,14.809074
15454,2019-11-18 23:00:00-03:00,135.283220,135.283220,-45.283220,-45.283220,197.464342,14.804576


In [13]:
colunas_desejadas = ['zenith', 'azimuth']
coluna = ['tracker_theta']
df_mes[colunas_desejadas] = solpos_mes[colunas_desejadas]
df_mes[coluna] = thetas_mes[coluna]
display(df_mes)


,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,Temperature,DNI,GHI,...,Surface Albedo,Cloud Type,Fill Flag,datetime,datetime_utc,datetime_local,Angulo,zenith,azimuth,tracker_theta
9696,9696,2019,7,22,0,0,0,14.6,0,0,...,0.15,7,0,2019-07-22 00:00:00,2019-07-22 00:00:00+00:00,2019-07-22 00:00:00-03:00,15,175.603772,223.818631,0.0
9697,9697,2019,7,22,0,30,0,14.5,0,0,...,0.15,7,0,2019-07-22 00:30:00,2019-07-22 00:30:00+00:00,2019-07-22 00:30:00-03:00,15,174.914320,128.305934,0.0
9698,9698,2019,7,22,1,0,0,14.3,0,0,...,0.15,7,0,2019-07-22 01:00:00,2019-07-22 01:00:00+00:00,2019-07-22 01:00:00-03:00,15,168.638525,104.161072,0.0
9699,9699,2019,7,22,1,30,0,14.3,0,0,...,0.15,7,0,2019-07-22 01:30:00,2019-07-22 01:30:00+00:00,2019-07-22 01:30:00-03:00,15,161.868768,96.488681,0.0
9700,9700,2019,7,22,2,0,0,14.2,0,0,...,0.15,7,0,2019-07-22 02:00:00,2019-07-22 02:00:00+00:00,2019-07-22 02:00:00-03:00,15,155.012959,92.182515,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15451,15451,2019,11,18,21,30,0,17.3,0,0,...,0.13,3,8,2019-11-18 21:30:00,2019-11-18 21:30:00+00:00,2019-11-18 21:30:00-03:00,60,124.927836,221.848110,0.0
15452,15452,2019,11,18,22,0,0,17.2,0,0,...,0.13,6,8,2019-11-18 22:00:00,2019-11-18 22:00:00+00:00,2019-11-18 22:00:00-03:00,15,129.190369,214.765982,0.0
15453,15453,2019,11,18,22,30,0,17.2,0,0,...,0.13,6,8,2019-11-18 22:30:00,2019-11-18 22:30:00+00:00,2019-11-18 22:30:00-03:00,15,132.701519,206.627335,0.0
15454,15454,2019,11,18,23,0,0,17.1,0,0,...,0.13,6,8,2019-11-18 23:00:00,2019-11-18 23:00:00+00:00,2019-11-18 23:00:00-03:00,15,135.283220,197.464342,0.0


Tratamento dos dados: Criar Hour_sine e Hour_cosine e normalizar os outros dados

In [14]:
from sklearn.preprocessing import MinMaxScaler

def calcula_seno(hora,minuto):
    hour_sine = np.sin((2*np.pi*(hora*60+minuto))/1440)
    return hour_sine

def calcula_cosseno(hora,minuto):
    hour_cosine = np.cos((2*np.pi*(hora*60+minuto))/1440)
    return hour_cosine

senos = []
cossenos = []

for i, row in df_mes.iterrows(): # row representa cada linha do df
    hour_sine = calcula_seno(row['Hour'],row['Minute'])
    senos.append(hour_sine)
    hour_cosine = calcula_cosseno(row['Hour'],row['Minute'])
    cossenos.append(hour_cosine)

df_mes['Hour Sine'] = senos
df_mes['Hour Cosine'] = cossenos




cols = ['Unnamed: 0','Year','Day','Hour','Minute','Temperature','Cloud Type','Fill Flag','Surface Albedo','datetime_utc','datetime_local','zenith']
df_mes = df_mes.drop(cols,axis=1)
display(df_mes)

,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,Temperature,DNI,GHI,...,Fill Flag,datetime,datetime_utc,datetime_local,Angulo,zenith,azimuth,tracker_theta,Hour Sine,Hour Cosine
9696,9696,2019,7,22,0,0,0,14.6,0,0,...,0,2019-07-22 00:00:00,2019-07-22 00:00:00+00:00,2019-07-22 00:00:00-03:00,15,175.603772,223.818631,0.0,0.000000,1.000000
9697,9697,2019,7,22,0,30,0,14.5,0,0,...,0,2019-07-22 00:30:00,2019-07-22 00:30:00+00:00,2019-07-22 00:30:00-03:00,15,174.914320,128.305934,0.0,0.130526,0.991445
9698,9698,2019,7,22,1,0,0,14.3,0,0,...,0,2019-07-22 01:00:00,2019-07-22 01:00:00+00:00,2019-07-22 01:00:00-03:00,15,168.638525,104.161072,0.0,0.258819,0.965926
9699,9699,2019,7,22,1,30,0,14.3,0,0,...,0,2019-07-22 01:30:00,2019-07-22 01:30:00+00:00,2019-07-22 01:30:00-03:00,15,161.868768,96.488681,0.0,0.382683,0.923880
9700,9700,2019,7,22,2,0,0,14.2,0,0,...,0,2019-07-22 02:00:00,2019-07-22 02:00:00+00:00,2019-07-22 02:00:00-03:00,15,155.012959,92.182515,0.0,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15451,15451,2019,11,18,21,30,0,17.3,0,0,...,8,2019-11-18 21:30:00,2019-11-18 21:30:00+00:00,2019-11-18 21:30:00-03:00,60,124.927836,221.848110,0.0,-0.608761,0.793353
15452,15452,2019,11,18,22,0,0,17.2,0,0,...,8,2019-11-18 22:00:00,2019-11-18 22:00:00+00:00,2019-11-18 22:00:00-03:00,15,129.190369,214.765982,0.0,-0.500000,0.866025
15453,15453,2019,11,18,22,30,0,17.2,0,0,...,8,2019-11-18 22:30:00,2019-11-18 22:30:00+00:00,2019-11-18 22:30:00-03:00,15,132.701519,206.627335,0.0,-0.382683,0.923880
15454,15454,2019,11,18,23,0,0,17.1,0,0,...,8,2019-11-18 23:00:00,2019-11-18 23:00:00+00:00,2019-11-18 23:00:00-03:00,15,135.283220,197.464342,0.0,-0.258819,0.965926


,Month,DHI,DNI,GHI,Solar Zenith Angle,Wind Speed,datetime,Angulo,azimuth,tracker_theta,Hour Sine,Hour Cosine
9696,7,0,0,0,175.60,2.0,2019-07-22 00:00:00,15,223.818631,0.0,0.000000,1.000000
9697,7,0,0,0,174.91,1.8,2019-07-22 00:30:00,15,128.305934,0.0,0.130526,0.991445
9698,7,0,0,0,168.63,1.7,2019-07-22 01:00:00,15,104.161072,0.0,0.258819,0.965926
9699,7,0,0,0,161.86,1.8,2019-07-22 01:30:00,15,96.488681,0.0,0.382683,0.923880
9700,7,0,0,0,155.01,1.8,2019-07-22 02:00:00,15,92.182515,0.0,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
15451,11,0,0,0,124.92,3.9,2019-11-18 21:30:00,60,221.848110,0.0,-0.608761,0.793353
15452,11,0,0,0,129.18,3.9,2019-11-18 22:00:00,15,214.765982,0.0,-0.500000,0.866025
15453,11,0,0,0,132.69,3.9,2019-11-18 22:30:00,15,206.627335,0.0,-0.382683,0.923880
15454,11,0,0,0,135.28,3.8,2019-11-18 23:00:00,15,197.464342,0.0,-0.258819,0.965926


In [15]:
df_mes.to_csv('df_mes.csv', index=False)